In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proj1"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Deep Learning/Proj1


In [3]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch import Tensor
from torch import nn
from torch import cuda

import dlc_practical_prologue as prologue

from models import *
from helpers import *

In [4]:
nb_samples = 1000
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(nb_samples)

# work on GPU if device is available
device_name = "cuda" if torch.cuda.is_available() else "cpu" 
device = torch.device(device_name)
print("Using device " + device_name)
train_input, train_target, train_classes, test_input, test_target, test_classes = train_input.to(device), train_target.to(device), train_classes.to(device), test_input.to(device), test_target.to(device), test_classes.to(device)

print('train_input', train_input.size(), 'train_classes', train_classes.size(), 'train_target', train_target.size())
print('test_input', test_input.size(), 'test_classes', test_classes.size(), 'test_target', test_target.size())

Using device cpu
train_input torch.Size([1000, 2, 14, 14]) train_classes torch.Size([1000, 2]) train_target torch.Size([1000])
test_input torch.Size([1000, 2, 14, 14]) test_classes torch.Size([1000, 2]) test_target torch.Size([1000])


In [4]:
nb_hidden = 200
nb_epochs = 25
lr = 1e-3
mini_batch_size = 50

shallow = ShallowModel(nb_hidden, nb_epochs, lr, mini_batch_size).to(device)
acc_losses, train_accuracy, test_accuracy, train_error, test_error = shallow.train_model(train_input, train_target, test_input, test_target)
test_accuracy

NameError: ignored

In [ ]:
nb_hidden = 200
nb_epochs = 100
lr = 1e-3
mini_batch_size = 25
auxiliary_loss_weight = 0.5

deep = DeepModel(nb_hidden, nb_epochs, lr, mini_batch_size, auxiliary_loss_weight).to(device)
acc_losses, train_accuracy, test_accuracy, train_error, test_error = deep.train_model(train_input, train_classes, train_target, test_input, test_target)
test_accuracy

In [ ]:
nb_hidden = 200
nb_epochs = 25
lr = 1e-1
mini_batch_size = 1
auxiliary_loss_weight = 0.5

deep_ws = DeepModelWS(nb_hidden, nb_epochs, lr, mini_batch_size, auxiliary_loss_weight).to(device)
acc_losses, train_accuracy, test_accuracy, train_error, test_error = deep_ws.train_model(train_input, train_classes, train_target, test_input, test_target)
test_accuracy

In [ ]:
torch.max(test_accuracy).item()

In [5]:
# parameters to be tested
optimizers = [optim.SGD, optim.Adam]
nb_epochs = [5, 25, 100]
lrs = [1e-4,1e-3,1e-2]
mini_batch_sizes = [1,250,1000]
nb_hiddens = [100,300,500] 
auxiliary_loss_weights = [0,0.5,1]
test_accuracies_deep = torch.zeros(len(optimizers), len(nb_epochs), len(lrs), len(mini_batch_sizes), len(nb_hiddens), len(auxiliary_loss_weights))
test_accuracies_deep_ws = torch.zeros(len(optimizers), len(nb_epochs), len(lrs), len(mini_batch_sizes), len(nb_hiddens), len(auxiliary_loss_weights))

In [ ]:
for i,optimizer in enumerate(optimizers):
  for j,nb_epoch in enumerate(nb_epochs):
    for k,lr in enumerate(lrs):
      for l,mini_batch_size in enumerate(mini_batch_sizes):
        for m,nb_hidden in enumerate(nb_hiddens):
          for n,auxiliary_loss_weight in enumerate(auxiliary_loss_weights):
            deep = DeepModel(nb_hidden, nb_epoch, lr, mini_batch_size, auxiliary_loss_weight, optimizer).to(device)
            acc_losses, train_accuracy, test_accuracy, train_error, test_error = deep.train_model(train_input, train_classes, train_target, test_input, test_target)

In [ ]:
for i,optimizer in enumerate(optimizers):
  for j,nb_epoch in enumerate(nb_epochs):
    for k,lr in enumerate(lrs):
      for l,mini_batch_size in enumerate(mini_batch_sizes):
        for m,nb_hidden in enumerate(nb_hiddens):
          for n,auxiliary_loss_weight in enumerate(auxiliary_loss_weights):
            deep_ws = DeepModelWS(nb_hidden, nb_epoch, lr, mini_batch_size, auxiliary_loss_weight, optimizer).to(device)
            acc_losses, train_accuracy, test_accuracy, train_error, test_error = deep_ws.train_model(train_input, train_classes, train_target, test_input, test_target)
            test_accuracies_deep_ws[i,j,k,l,m,n] = torch.max(test_accuracy).item()

In [ ]:
kfor j,nb_epoch in enumerate(nb_epochs):
  print(j, nb_epoch)

In [1]:
test_accuracies_deep_ws

NameError: ignored